In [1]:
import numpy as np
import statsmodels
from causalimpact import CausalImpact
import pandas as pd
from scipy import stats
import matplotlib
import seaborn as sns
import lifelines

In [622]:
dfc = pd.DataFrame(pd.read_pickle('compliance.pkl'))

In [623]:
dfb = pd.DataFrame(pd.read_pickle('survey.pkl'))

In [624]:
df = pd.DataFrame(pd.read_csv('compliance_data.csv'))

In [625]:
s1Start = "2016-08-23"
s1End = '2016-12-10'
s2Start = "2016-12-11"
s2End = '2019-04-22'

In [626]:
df = df[['participID','date','week','complyPercent']]

In [627]:
df["Spring"] = 0
df.loc[((df.date >= s1Start)&(df.date <= s1End )) , 'Spring'] = 1
df.loc[((df.date >= s1Start)) , 'Start'] = 1
df = df[df['Start'] == 1]

dfF = df[df['Spring'] == 1]
groupedF =dfF.groupby('participID')
grouped =df.groupby('participID')
df['week_after_intervention'] = ((pd.to_datetime(df['date']) - pd.to_datetime(s2Start)).dt.days / 7)

In [628]:
grouped = grouped['complyPercent'].mean()
participI  =  grouped.index
mean = grouped.values

dict = {'participID': participI, 'semester_compliance': mean}  
    
dfF = pd.DataFrame(dict)

In [629]:
df = df.merge(dfF,right_on = 'participID',left_on = 'participID')

In [630]:
df = df.merge(dfb,right_on = 'id',left_on = 'participID')

In [631]:
df['left_study'] = 0
df.loc[(df.date > s2Start)& (df.date < s2End),'left_study'] = 1


In [632]:
df = df.groupby('participID').tail(1)


In [633]:
df.left_study.value_counts()

1    306
0    166
Name: left_study, dtype: int64

In [634]:
df = df[df.week_after_intervention > 0]

In [635]:
df3 = df
df3 = df3.groupby('participID').tail(1)
df3 = df3[['week','semester_compliance','left_study']]
df3 = df3.dropna()
df2 =df
df2.Extraversion_2 = df2.Extraversion_2.fillna(df2.Extraversion_1)
df2.Agreeableness_2 = df2.Agreeableness_2.fillna(df2.Agreeableness_1)
df2.Openness_2 = df2.Openness_2.fillna(df2.Openness_1)
df2.Neuroticism_2 = df2.Neuroticism_2.fillna(df2.Neuroticism_1)
df2.Conscientiousness_2 = df2.Conscientiousness_2.fillna(df2.Conscientiousness_1)
df2 = df2.groupby('participID').tail(1)
df4 = df2[['week','semester_compliance','left_study','Extraversion_2','Agreeableness_2','Conscientiousness_2','Openness_2','Neuroticism_2','gender_1','race_1','parentincome_1']]
df2 = df2[['week','week_after_intervention','semester_compliance','left_study','Extraversion_2','Agreeableness_2','Conscientiousness_2','Openness_2','Neuroticism_2','gender_1','parentincome_1','race_1']]
df2 = df2.dropna()
df4 = df4.dropna()

In [636]:
from lifelines import CoxPHFitter

In [637]:
df2.loc[df2['semester_compliance'] >= 80, 'Satisfactory' ] = 1
df2.loc[df2['semester_compliance'] < 80, 'Satisfactory' ] = 0

In [638]:
bad = df2[df2['semester_compliance'] < 80]

In [639]:
df.head()

,participID,date,week,complyPercent,Spring,Start,week_after_intervention,semester_compliance,id,stepChallenge,...,Conscientiousness_1,Conscientiousness_2,Conscientiousness_6,Openness_1,Openness_2,Openness_6,Neuroticism_1,Neuroticism_2,Neuroticism_6,left_study
481,39WN3Y,2017-12-17,93,4.0,0,1.0,53.000000,18.808333,39WN3Y,1.0,...,2.777778,2.777778,2.333333,2.9,2.900000,3.111111,3.625,3.625,3.750,1
847,3KSZ8V,2017-07-23,103,96.0,0,1.0,32.000000,96.459459,3KSZ8V,1.0,...,4.333333,4.666667,NaN,3.7,3.666667,NaN,2.375,2.125,NaN,1
1462,3LZ4Q3,2018-04-29,145,94.0,0,1.0,72.000000,88.659054,3LZ4Q3,1.0,...,4.000000,4.000000,4.444445,3.0,3.444444,3.111111,3.375,3.875,4.000,1
1700,3LZ8FZ,2017-04-17,90,50.0,0,1.0,18.142857,72.489362,3LZ8FZ,1.0,...,4.000000,3.444444,NaN,3.7,4.222222,NaN,3.250,3.500,NaN,1
2458,3LZGLG,2018-09-10,164,98.0,0,1.0,91.142857,93.697051,3LZGLG,1.0,...,4.555555,4.222222,4.111111,3.2,3.666667,4.000000,3.250,2.375,2.625,1


In [640]:
dfT = df2[['week','week_after_intervention','semester_compliance','left_study']]

In [641]:


cph = CoxPHFitter()
cph.fit(df2, duration_col='week_after_intervention', event_col='left_study' , strata = ['gender_1','race_1','parentincome_1'])

cph.print_summary()

<lifelines.CoxPHFitter: fitted with 403 total observations, 112 right-censored observations>
             duration col = 'week_after_intervention'
                event col = 'left_study'
                   strata = ['gender_1', 'race_1', 'parentincome_1']
   number of observations = 403
number of events observed = 291
   partial log-likelihood = -373.19
         time fit was run = 2019-11-01 18:34:04 UTC

---
                      coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
week                 -0.05       0.95       0.00            -0.06            -0.04                 0.95                 0.96
semester_compliance  -0.01       0.99       0.00            -0.02            -0.00                 0.98                 1.00
Extraversion_2       -0.23       0.80       0.12            -0.46             0.00                 0.63                 1.00
Agreeableness_2      -0.47       0.63       0.16            -0.79            -0.15    

In [642]:
dfT.head()

,week,week_after_intervention,semester_compliance,left_study
481,93,53.000000,18.808333,1
847,103,32.000000,96.459459,1
1462,145,72.000000,88.659054,1
1700,90,18.142857,72.489362,1
2458,164,91.142857,93.697051,1
